In [1]:
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf
import org.apache.spark.sql.SQLContext
import org.apache.spark.sql.functions._

In [2]:
val conf = new SparkConf().setAppName("lab02_copy").setMaster("yarn")
val start = System.currentTimeMillis()
val sc = new SparkContext(conf)
println(s"Context created in ${(System.currentTimeMillis() - start) / 1e3} secs")

Context created in 12.626 secs


conf = org.apache.spark.SparkConf@cb9e0d2
start = 1632520241346
sc = org.apache.spark.SparkContext@28eafd14


org.apache.spark.SparkContext@28eafd14

In [3]:
val sqlContext = new SQLContext(sc);

sqlContext = org.apache.spark.sql.SQLContext@33bd598c


org.apache.spark.sql.SQLContext@33bd598c

In [4]:
val users = sqlContext.read.json("/labs/laba02/autousers.json")
                            .select(explode('autousers))
                            .rdd.map(r => r(0))
                            .collect.toList

users = List(100341861572, 100473724387, 100528753939, 101182896723, 101818209438, 101886368239, 102535398052, 103624582558, 103753741251, 104091409587, 104619031972, 104636088851, 104684233907, 104979586580, 105019547390, 105036954964, 105393450179, 105489980463, 10565067157, 105794662484, 106211111663, 106240201246, 107321870895, 107404936340, 107468636398, 107691294975, 107708325982, 10775836053, 107782097795, 107937872660, 108037887630, 108064895903, 10823573029, 10855721781, 108844671060, 10895608725, 10915265653, 109685047727, 110125660324, 110378823806, 111090583023, 111267576526, 111422370446, 111499688691, 112012631455, 112922985363, 112961302852, 113539278415, 113854647588, 113860126398, 114013491252, 114062617630, 114372912483, 114379300916, 114380549796, 114462170...


List(100341861572, 100473724387, 100528753939, 101182896723, 101818209438, 101886368239, 102535398052, 103624582558, 103753741251, 104091409587, 104619031972, 104636088851, 104684233907, 104979586580, 105019547390, 105036954964, 105393450179, 105489980463, 10565067157, 105794662484, 106211111663, 106240201246, 107321870895, 107404936340, 107468636398, 107691294975, 107708325982, 10775836053, 107782097795, 107937872660, 108037887630, 108064895903, 10823573029, 10855721781, 108844671060, 10895608725, 10915265653, 109685047727, 110125660324, 110378823806, 111090583023, 111267576526, 111422370446, 111499688691, 112012631455, 112922985363, 112961302852, 113539278415, 113854647588, 113860126398, 114013491252, 114062617630, 114372912483, 114379300916, 114380549796, 114462170...

In [5]:
val logs = sqlContext
                .read
                .format("csv")
                .option("delimiter", "\t")
                .load("/labs/laba02/logs/*")

logs = [_c0: string, _c1: string ... 1 more field]


[_c0: string, _c1: string ... 1 more field]

In [6]:
import org.apache.spark.sql.{Dataset, Column}

In [7]:
import org.apache.spark.sql.functions.udf
import scala.util.Try
import java.net.URL
import java.net.URLDecoder.decode

In [8]:
val decode_url = udf { (url: String) => Try(new URL(decode(url, "UTF-8")).getHost).toOption}

decode_url = UserDefinedFunction(<function1>,StringType,Some(List(StringType)))


UserDefinedFunction(<function1>,StringType,Some(List(StringType)))

In [9]:
val df = logs
            .select(col("_c0").alias("UID"), col("_c1").alias("Timestamp"), col("_c2").alias("URL"))
            .na.drop(List("UID", "URL"))
            .filter('URL.contains("http"))
            .withColumn("URL", decode_url(col("URL")))
//             .filter('UID != "--" &&
//                     'UID != "-" &&
//                     'URL != "--" &&
//                     'URL != "-")

df = [UID: string, Timestamp: string ... 1 more field]


[UID: string, Timestamp: string ... 1 more field]

In [10]:
val filtered_df = df.withColumn("url", regexp_replace('URL, "^www.", "")).dropDuplicates

filtered_df = [UID: string, Timestamp: string ... 1 more field]


[UID: string, Timestamp: string ... 1 more field]

In [11]:
val newCol = when(filtered_df("UID").isin(users:_*), 1).otherwise(0)

val stat = filtered_df.withColumn("Auto-flag", newCol).drop("UID", "Timestamp")

newCol = CASE WHEN (UID IN (100341861572, 100473724387, 100528753939, 101182896723, 101818209438, 101886368239, 102535398052, 103624582558, 103753741251, 104091409587, 104619031972, 104636088851, 104684233907, 104979586580, 105019547390, 105036954964, 105393450179, 105489980463, 10565067157, 105794662484, 106211111663, 106240201246, 107321870895, 107404936340, 107468636398, 107691294975, 107708325982, 10775836053, 107782097795, 107937872660, 108037887630, 108064895903, 10823573029, 10855721781, 108844671060, 10895608725, 10915265653, 109685047727, 110125660324, 110378823806, 111090583023, 111267576526, 111422370446, 111499688691, 112012631455, 112922985363, 112961302852, 113539278415, 113854647588, 113860126398, 114013491252, 114062617630, 114372912483, 1143...


CASE WHEN (UID IN (100341861572, 100473724387, 100528753939, 101182896723, 101818209438, 101886368239, 102535398052, 103624582558, 103753741251, 104091409587, 104619031972, 104636088851, 104684233907, 104979586580, 105019547390, 105036954964, 105393450179, 105489980463, 10565067157, 105794662484, 106211111663, 106240201246, 107321870895, 107404936340, 107468636398, 107691294975, 107708325982, 10775836053, 107782097795, 107937872660, 108037887630, 108064895903, 10823573029, 10855721781, 108844671060, 10895608725, 10915265653, 109685047727, 110125660324, 110378823806, 111090583023, 111267576526, 111422370446, 111499688691, 112012631455, 112922985363, 112961302852, 113539278415, 113854647588, 113860126398, 114013491252, 114062617630, 114372912483, 1143...

In [12]:
val domains = stat
                .groupBy("url")
                .count
                .withColumnRenamed("count", "domain")
                .withColumnRenamed("url", "url_domains")

domains = [url_domains: string, domain: bigint]


[url_domains: string, domain: bigint]

In [13]:
val domains_n_drivers = stat
                .groupBy("url")
                .sum("Auto-flag")
                .withColumnRenamed("sum(Auto-flag)", "domain_x_driver")

domains_n_drivers = [url: string, domain_x_driver: bigint]


[url: string, domain_x_driver: bigint]

In [14]:
val drivers_size = stat
                    .filter(col("Auto-flag") === 1)
                    .count

drivers_size = 313515


313515

In [36]:
import org.apache.spark.sql.types.{DecimalType, StringType}

In [37]:
val res = domains_n_drivers
                .join(
                    domains,
                    domains("url_domains") === domains_n_drivers("url"),
                    "inner"
                    )
                .withColumn("domain_x_driver",
                            round(pow('domain_x_driver, 2)
                            / ('domain * drivers_size), 20))
                                .drop("domain", "url_domains")
                .withColumnRenamed("domain_x_driver", "relevance")
                .orderBy('relevance.desc, 'url.asc)
                .withColumn("relevance", 'relevance
                                                    .cast(DecimalType(21, 20))
                                                    .cast(StringType))

res = [url: string, relevance: string]


[url: string, relevance: string]

In [38]:
res.filter('relevance.isNull).count

0

In [40]:
res.limit(200).coalesce(1)
      .write
      .option("header", "false")
      .option("sep", "\t")
      .mode("overwrite")
      .csv("laba02_domains.txt")

lastException: Throwable = null


In [41]:
// sc.stop